In [1]:
from keras.models import Sequential
from keras.layers import Dense

import pandas as pd
import numpy
import tensorflow as tf

# seed 값 설정
numpy.random.seed(3)
tf.random.set_seed(3)

Using TensorFlow backend.


In [3]:
# 데이터 불러오기
df = pd.read_csv('data/MatchTimelinesFirst15.csv')
df.head(10)

,Unnamed: 0,matchId,blue_win,blueGold,blueMinionsKilled,blueJungleMinionsKilled,blueAvgLevel,redGold,redMinionsKilled,redJungleMinionsKilled,redAvgLevel,blueChampKills,blueHeraldKills,blueDragonKills,blueTowersDestroyed,redChampKills,redHeraldKills,redDragonKills,redTowersDestroyed
0,0,3.493251e+09,0,24575.0,349.0,89.0,8.6,25856.0,346.0,80.0,9.2,6.0,1.0,0.0,1.0,12.0,2.0,0.0,1.0
1,1,3.464936e+09,0,27210.0,290.0,36.0,9.0,28765.0,294.0,92.0,9.4,20.0,0.0,0.0,0.0,19.0,2.0,0.0,0.0
2,2,3.428426e+09,1,32048.0,346.0,92.0,9.4,25305.0,293.0,84.0,9.4,17.0,3.0,0.0,0.0,11.0,0.0,0.0,4.0
3,3,3.428347e+09,0,20261.0,223.0,60.0,8.2,30429.0,356.0,107.0,9.4,7.0,0.0,0.0,3.0,16.0,3.0,0.0,0.0
4,4,3.428351e+09,1,30217.0,376.0,110.0,9.8,23889.0,334.0,60.0,8.8,16.0,3.0,0.0,0.0,8.0,0.0,0.0,2.0
5,5,3.494459e+09,1,25470.0,362.0,82.0,9.2,22856.0,319.0,86.0,8.8,9.0,1.0,0.0,0.0,7.0,1.0,0.0,0.0
6,6,3.463321e+09,1,25391.0,350.0,96.0,9.2,23236.0,345.0,80.0,8.6,8.0,2.0,0.0,0.0,5.0,1.0,0.0,1.0
7,7,3.463280e+09,0,28946.0,367.0,65.0,9.2,26528.0,311.0,63.0,8.8,15.0,2.0,0.0,1.0,14.0,1.0,0.0,1.0
8,8,3.462719e+09,1,26928.0,379.0,80.0,9.6,23065.0,362.0,95.0,9.2,11.0,1.0,0.0,0.0,5.0,2.0,0.0,1.0
9,9,3.462734e+09,1,26079.0,302.0,106.0,9.0,27275.0,347.0,91.0,9.4,9.0,1.0,0.0,1.0,10.0,2.0,0.0,1.0


In [4]:
# 데이터 분석
df.info()
# blue_win을 제외한 모든 데이터가 float 타입
# blue_win은 class 이므로 모든 데이터는 float 타입이라고 볼 수 있음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48651 entries, 0 to 48650
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               48651 non-null  int64  
 1   matchId                  48651 non-null  float64
 2   blue_win                 48651 non-null  int64  
 3   blueGold                 48651 non-null  float64
 4   blueMinionsKilled        48651 non-null  float64
 5   blueJungleMinionsKilled  48651 non-null  float64
 6   blueAvgLevel             48651 non-null  float64
 7   redGold                  48651 non-null  float64
 8   redMinionsKilled         48651 non-null  float64
 9   redJungleMinionsKilled   48651 non-null  float64
 10  redAvgLevel              48651 non-null  float64
 11  blueChampKills           48651 non-null  float64
 12  blueHeraldKills          48651 non-null  float64
 13  blueDragonKills          48651 non-null  float64
 14  blueTowersDestroyed   

In [6]:
df.describe()
#blueDragonKills와 redDragonKills는 모든 값이 0이기 때문에 의미없는 데이터임.
#matchId는 게임번호이고 Unnamed는 인덱스이므로 승부예측과는 관련없는 데이터임.

,Unnamed: 0,matchId,blue_win,blueGold,blueMinionsKilled,blueJungleMinionsKilled,blueAvgLevel,redGold,redMinionsKilled,redJungleMinionsKilled,redAvgLevel,blueChampKills,blueHeraldKills,blueDragonKills,blueTowersDestroyed,redChampKills,redHeraldKills,redDragonKills,redTowersDestroyed
count,48651.00000,4.865100e+04,48651.000000,48651.000000,48651.000000,48651.000000,48651.000000,48651.000000,48651.000000,48651.000000,48651.000000,48651.000000,48651.000000,48651.0,48651.000000,48651.000000,48651.000000,48651.0,48651.000000
mean,24325.00000,3.486083e+09,0.505416,26349.160572,335.859140,83.136585,9.172251,26203.028591,336.693984,84.692997,9.174759,11.013155,1.316355,0.0,0.794084,10.747693,1.287188,0.0,0.891657
std,14044.47831,2.812307e+07,0.499976,2777.299868,32.673096,14.794947,0.413735,2754.709425,33.278962,14.902326,0.421396,4.532898,0.956784,0.0,1.054556,4.453358,0.952450,0.0,1.111205
min,0.00000,2.544365e+09,0.000000,15930.000000,120.000000,0.000000,5.400000,11133.000000,14.000000,1.000000,2.400000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
25%,12162.50000,3.475554e+09,0.000000,24401.500000,315.000000,73.000000,9.000000,24301.000000,316.000000,75.000000,9.000000,8.000000,1.000000,0.0,0.000000,8.000000,1.000000,0.0,0.000000
50%,24325.00000,3.494380e+09,1.000000,26167.000000,337.000000,83.000000,9.200000,26029.000000,338.000000,84.000000,9.200000,11.000000,1.000000,0.0,1.000000,10.000000,1.000000,0.0,1.000000
75%,36487.50000,3.506802e+09,1.000000,28126.000000,358.000000,92.000000,9.400000,27940.000000,359.000000,94.000000,9.400000,14.000000,2.000000,0.0,1.000000,14.000000,2.000000,0.0,1.000000
max,48650.00000,3.513125e+09,1.000000,39769.000000,455.000000,164.000000,10.800000,39999.000000,457.000000,160.000000,10.600000,50.000000,4.000000,0.0,11.000000,43.000000,4.000000,0.0,10.000000
